# Пример общего рабочего процесса обработки осциллограмм

Этот ноутбук демонстрирует возможную последовательность шагов при обработке коллекции осциллограмм с использованием ранее созданных модульных классов. Каждый этап может быть адаптирован, пропущен или выполнен независимо в соответствии с конкретными задачами.

**Этапы рабочего процесса:**
1.  **Обнаружение и сбор осциллограмм**: Используем `OscillogramFinder` для поиска и копирования файлов нужных типов из исходной директории (включая архивы) в рабочую директорию.
2.  **Анонимизация**: Применяем `DataAnonymizer` для удаления конфиденциальной информации и переименования файлов по хеш-сумме.
3.  **Каталогизация имен сигналов**: Используем `SignalNameManager` для создания каталога всех имен сигналов. (Опционально: их последующее переименование).
4.  **Фильтрация по активности**: Применяем `OscillogramActivityFilter` для отсеивания "пустых" или неактивных осциллограмм.
5.  **Генерация коэффициентов нормализации**: На основе активных осциллограмм создаем файл `norm.csv` с помощью `NormalizationCoefficientGenerator`.
6.  **Преобразование в CSV**: Конвертируем обработанные (анонимизированные, активные, нормализованные) осциллограммы в формат CSV с помощью `OscillogramToCsvConverter` для дальнейшего анализа или использования в ML.
7.  **Специализированный анализ (пример)**: Краткая демонстрация запуска одного из анализаторов, например, `SPEFAnalyzer` или `OvervoltageDetector`.

Все пути к файлам и директориям, а также конфигурации, являются примерами и должны быть адаптированы под реальные задачи.

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/`. Для демонстрации файлы копируются во временную рабочую директорию (`temp_workflow_notebook_demonstration/...`). Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json 
import csv
import zipfile 
import sys

# --- Пути к данным для тестирования ---
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
from core.oscillogram import Oscillogram
from filesystem.finder import OscillogramFinder, TYPE_OSC
from preprocessing.anonymizer import DataAnonymizer
from preprocessing.signal_names import SignalNameManager
from analysis.activity_filter import OscillogramActivityFilter, ChannelType
from normalization.normalization import NormalizationCoefficientGenerator, OscillogramNormalizer
from raw_to_csv.raw_to_csv import OscillogramToCsvConverter 
# from analysis.spef import SPEFAnalyzer # Uncomment if SPEF demo is fleshed out
# from analysis.overvoltage_detector import OvervoltageDetector # Uncomment if OV demo is fleshed out

base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
comtrade_samples_path = os.path.join(base_sample_data_dir, "comtrade_files")
archive_samples_path = os.path.join(base_sample_data_dir, "archives")
config_samples_path = os.path.join(base_sample_data_dir, "config_files")

# --- Основные директории для рабочего процесса в этом ноутбуке ---
notebook_base_workflow_dir = "temp_workflow_notebook_demonstration"
initial_source_dir_wf = os.path.join(notebook_base_workflow_dir, "00_initial_source")
staging_dir_found_wf = os.path.join(notebook_base_workflow_dir, "01_staging_found_oscillograms")
# staging_dir_anonymized_wf и staging_dir_active_wf будут созданы позже копированием
notebook_output_configs_dir_wf = os.path.join(notebook_base_workflow_dir, "output_and_configs_workflow")

if os.path.exists(notebook_base_workflow_dir):
    shutil.rmtree(notebook_base_workflow_dir)
os.makedirs(initial_source_dir_wf)
os.makedirs(staging_dir_found_wf) 
os.makedirs(notebook_output_configs_dir_wf)
print(f"Созданы базовые директории в: {notebook_base_workflow_dir}")

# --- Функция для копирования с проверкой ---
def copy_wf_sample(src_full_path, dest_dir):
    if os.path.exists(src_full_path):
        os.makedirs(dest_dir, exist_ok=True) # Убедимся, что директория существует
        shutil.copy(src_full_path, os.path.join(dest_dir, os.path.basename(src_full_path)))
        return True
    print(f"ОШИБКА: Исходный файл для workflow не найден: {src_full_path}")
    return False

# --- Наполнение initial_source_dir_wf ---
all_setup_ok = True
# Активная осциллограмма (из данных для ActivityFilter)
all_setup_ok &= copy_wf_sample(os.path.join(comtrade_samples_path, "for_activity_filter", "osc_af_active.cfg"), initial_source_dir_wf)
all_setup_ok &= copy_wf_sample(os.path.join(comtrade_samples_path, "for_activity_filter", "osc_af_active.dat"), initial_source_dir_wf)
# "Пустая" осциллограмма
all_setup_ok &= copy_wf_sample(os.path.join(comtrade_samples_path, "for_activity_filter", "osc_af_empty_stable.cfg"), initial_source_dir_wf)
all_setup_ok &= copy_wf_sample(os.path.join(comtrade_samples_path, "for_activity_filter", "osc_af_empty_stable.dat"), initial_source_dir_wf)
# Осциллограмма в поддиректории
workflow_subdir_path = os.path.join(initial_source_dir_wf, "workflow_subdir")
os.makedirs(workflow_subdir_path, exist_ok=True)
all_setup_ok &= copy_wf_sample(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.cfg"), workflow_subdir_path)
all_setup_ok &= copy_wf_sample(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.dat"), workflow_subdir_path)
# Архив
all_setup_ok &= copy_wf_sample(os.path.join(archive_samples_path, "archive_A.zip"), initial_source_dir_wf) # archive_A.zip содержит sample_arc_A.cfg/dat
# Текстовый файл
with open(os.path.join(initial_source_dir_wf, "readme.txt"), "w") as f: f.write("some info")

# Конфигурационные файлы, которые будут использоваться (копируем их в notebook_output_configs_dir_wf)
all_setup_ok &= copy_wf_sample(os.path.join(config_samples_path, "dict_analog_names_sample.json"), notebook_output_configs_dir_wf)
all_setup_ok &= copy_wf_sample(os.path.join(config_samples_path, "dict_discrete_names_sample.json"), notebook_output_configs_dir_wf)
all_setup_ok &= copy_wf_sample(os.path.join(config_samples_path, "activity_filter_config.json"), notebook_output_configs_dir_wf)

if not all_setup_ok: print("ВНИМАНИЕ: Не все файлы для демонстрации рабочего процесса были успешно настроены.")
else: print(f"Содержимое {initial_source_dir_wf} и {notebook_output_configs_dir_wf} подготовлено.")

## Этап 1: Обнаружение и сбор осциллограмм (`OscillogramFinder`)
Копируем только файлы COMTRADE (.cfg/.dat) из `initial_source_dir` (включая архивы) в `staging_dir_found`.
Сохраняем структуру директорий и используем хеши для избежания дубликатов.

In [ ]:
finder_wf = OscillogramFinder(is_print_message=True, show_progress_bars=True)
initial_hashes_path_wf = os.path.join(notebook_output_configs_dir_wf, "_hash_table_workflow.json") 
# Finder будет создавать _hash_table.json в dest_dir (staging_dir_found_wf),
# но мы можем указать ему использовать существующий или сохранить копию в notebook_output_configs_dir_wf.
# Для этого демо, мы позволим ему создать его в staging_dir_found_wf.
# initial_hashes_path_wf здесь будет использован для отображения, если нужно.

ft_flags = {type_enum: False for type_enum in TYPE_OSC} 
ft_flags[TYPE_OSC.COMTRADE_CFG_DAT] = True
ft_flags[TYPE_OSC.ARCH_ZIP] = True
ft_flags[TYPE_OSC.ARCH_7Z] = True 
ft_flags[TYPE_OSC.ARCH_RAR] = True 

copied_count_wf = finder_wf.copy_new_oscillograms(
    source_dir=initial_source_dir_wf,
    dest_dir=staging_dir_found_wf,
    preserve_dir_structure=True,
    use_hashes=True,
    file_type_flags=ft_flags,
    max_archive_depth=1, 
    show_progress_method=True
)
print(f"\nЭтап 1: Скопировано {copied_count_wf} осциллограмм в {staging_dir_found_wf}")
print(f"Содержимое {staging_dir_found_wf}:")
for root, dirs, files in os.walk(staging_dir_found_wf):
    level = root.replace(staging_dir_found_wf, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files: print(f"{sub_indent}{f_name}")

## Этап 2: Анонимизация (`DataAnonymizer`)
Анонимизируем файлы в `staging_dir_found` и сохраняем их в `staging_dir_anonymized`.
DataAnonymizer сам переименовывает файлы в хеши и кладет их в ту же директорию, где нашел.
Поэтому для чистоты эксперимента, сначала скопируем файлы в `staging_dir_anonymized`, а потом анонимизируем их там.

In [ ]:
# Создаем директорию для анонимизированных файлов
staging_dir_anonymized_wf = os.path.join(notebook_base_workflow_dir, "02_staging_anonymized")
if os.path.exists(staging_dir_anonymized_wf): shutil.rmtree(staging_dir_anonymized_wf)
shutil.copytree(staging_dir_found_wf, staging_dir_anonymized_wf, dirs_exist_ok=True) 
print(f"Скопированы файлы из {staging_dir_found_wf} в {staging_dir_anonymized_wf} для анонимизации.")

anonymizer_wf = DataAnonymizer(verbose_logging=True, show_progress_bars=True)
anonymizer_wf.anonymize_directory(staging_dir_anonymized_wf) 

print(f"\nЭтап 2: Анонимизация завершена в {staging_dir_anonymized_wf}.")
print(f"Содержимое {staging_dir_anonymized_wf} (файлы должны быть переименованы в хеши):")
for root, dirs, files in os.walk(staging_dir_anonymized_wf):
    level = root.replace(staging_dir_anonymized_wf, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files: print(f"{sub_indent}{f_name}")

## Этап 3: Каталогизация имен сигналов (`SignalNameManager`)
Создадим каталог имен сигналов из анонимизированных файлов.

In [ ]:
sm_wf = SignalNameManager(verbose_logging=False, show_progress_bars=True)
catalog_path_wf = os.path.join(notebook_output_configs_dir_wf, "workflow_signal_catalog.csv")
sm_wf.find_signal_names(staging_dir_anonymized_wf, 
                        output_csv_path=catalog_path_wf, 
                        verbose_logging_method=True, 
                        show_progress_method=True)

if os.path.exists(catalog_path_wf):
    print("\nСодержимое каталога сигналов:")
    try: from IPython.display import display; display(pd.read_csv(catalog_path_wf))
    except ImportError: print(pd.read_csv(catalog_path_wf))

## Этап 4: Фильтрация по активности (`OscillogramActivityFilter`)
Отфильтруем "пустые" осциллограммы из анонимизированной директории. Результаты (список активных файлов) сохраним в CSV. Затем скопируем только активные файлы в `staging_dir_active_wf`.

In [ ]:
# Загружаем конфигурацию ActivityFilter из файла (скопированного в setup)
activity_filter_config_path_wf = os.path.join(notebook_output_configs_dir_wf, "activity_filter_config.json")
activity_filter_config_wf = {}
if os.path.exists(activity_filter_config_path_wf):
    with open(activity_filter_config_path_wf, 'r', encoding='utf-8') as f_cfg_af:
        activity_filter_config_wf = json.load(f_cfg_af)
else:
    print(f"ОШИБКА: Файл конфигурации {activity_filter_config_path_wf} не найден. Используется дефолтная конфигурация.")
    # Можно задать дефолтную конфигурацию здесь, если нужно
    activity_filter_config_wf = {'use_norm_osc': False, 'channels_to_analyze_patterns': ['U_Active', 'I_Stable', 'U_Archived']}
    # Дополнить дефолтную конфигурацию необходимыми ключами, если они отсутствуют
    if 'raw_signal_analysis' not in activity_filter_config_wf:
        activity_filter_config_wf['raw_signal_analysis'] = {
            'initial_window_check_periods': 1, 'h1_vs_hx_ratio_threshold_U': 2, 
            'h1_vs_hx_ratio_threshold_I': 2,'min_initial_h1_amplitude_for_rel_norm': 0.005,
            'thresholds_raw_current_relative': {'delta': 0.1, 'std_dev': 0.05},
            'thresholds_raw_voltage_relative': {'delta': 0.1, 'std_dev': 0.05}
        }
    if 'current_channel_id_patterns' not in activity_filter_config_wf: activity_filter_config_wf['current_channel_id_patterns'] = ['i ']
    if 'voltage_channel_id_patterns' not in activity_filter_config_wf: activity_filter_config_wf['voltage_channel_id_patterns'] = ['u ']

# На данном этапе у нас еще нет файла norm.csv для workflow, поэтому normalizer=None или use_norm_osc=False
activity_filter_config_wf['use_norm_osc'] = False 
activity_filter_wf = OscillogramActivityFilter(config=activity_filter_config_wf, normalizer=None, show_progress_bars=True)

active_files_report_path_wf = os.path.join(notebook_output_configs_dir_wf, "workflow_active_files.csv")

print(f"\nФильтрация по активности в {staging_dir_anonymized_wf}...")
activity_filter_wf.filter_directory(staging_dir_anonymized_wf, active_files_report_path_wf, show_progress=True)

# Создаем директорию для активных файлов
staging_dir_active_wf = os.path.join(notebook_base_workflow_dir, "03_staging_active_oscillograms")
if os.path.exists(staging_dir_active_wf): shutil.rmtree(staging_dir_active_wf)
os.makedirs(staging_dir_active_wf)

if os.path.exists(active_files_report_path_wf):
    active_files_df = pd.read_csv(active_files_report_path_wf)
    if not active_files_df.empty and 'active_files' in active_files_df.columns:
        print(f"\nКопирование {len(active_files_df)} активных файлов в {staging_dir_active_wf}...")
        for _, row in active_files_df.iterrows():
            active_cfg_name = row['active_files'] 
            for root, _, files in os.walk(staging_dir_anonymized_wf):
                if active_cfg_name in files:
                    src_cfg_path = os.path.join(root, active_cfg_name)
                    src_dat_path = os.path.join(root, active_cfg_name[:-4] + ".dat")
                    
                    relative_dir = os.path.relpath(root, staging_dir_anonymized_wf)
                    dest_subdir_active = os.path.join(staging_dir_active_wf, relative_dir)
                    os.makedirs(dest_subdir_active, exist_ok=True)
                    
                    dest_cfg_path = os.path.join(dest_subdir_active, active_cfg_name)
                    dest_dat_path = os.path.join(dest_subdir_active, active_cfg_name[:-4] + ".dat")
                    
                    if os.path.exists(src_cfg_path): shutil.copy2(src_cfg_path, dest_cfg_path)
                    if os.path.exists(src_dat_path): shutil.copy2(src_dat_path, dest_dat_path)
                    break 
        print(f"Содержимое {staging_dir_active_wf} после копирования активных файлов:")
        for root, dirs, files in os.walk(staging_dir_active_wf):
            level = root.replace(staging_dir_active_wf, '').count(os.sep)
            indent = ' ' * 4 * (level); print(f"{indent}{os.path.basename(root)}/")
            sub_indent = ' ' * 4 * (level + 1)
            for f_name in files: print(f"{sub_indent}{f_name}")
    else:
        print("Активных файлов для копирования не найдено.")
else:
    print("Отчет об активных файлах не создан.")

## Этап 5: Генерация коэффициентов нормализации (`NormalizationCoefficientGenerator`)
Используем активные, анонимизированные осциллограммы из `staging_dir_active_wf` для генерации `norm_coeffs_workflow.csv`.

In [ ]:
workflow_norm_csv_path = os.path.join(notebook_output_configs_dir_wf, "norm_coeffs_workflow.csv") 

coeff_gen_wf = NormalizationCoefficientGenerator(
    osc_path=staging_dir_active_wf, # Директория с осциллограммами для анализа
    # prev_norm_csv_path=workflow_norm_csv_path, # Если хотим дописывать в существующий
    prev_norm_csv_path="", 
    bus=1,
    is_print_message=False # Управляется через tqdm внутри класса
)

print("\nЗапуск генерации коэффициентов нормализации...")
coeff_gen_wf.normalization(output_filename=workflow_norm_csv_path) # Указываем путь для сохранения

if os.path.exists(workflow_norm_csv_path):
    print(f"\nФайл коэффициентов '{workflow_norm_csv_path}' успешно создан.")
    try: from IPython.display import display; display(pd.read_csv(workflow_norm_csv_path).head())
    except ImportError: print(pd.read_csv(workflow_norm_csv_path).head())
else:
    print(f"\nОшибка: Файл '{workflow_norm_csv_path}' не был создан.")

## Этап 6: Преобразование в CSV (`OscillogramToCsvConverter`)
Конвертируем активные, анонимизированные осциллограммы в CSV, используя сгенерированные коэффициенты нормализации.

In [ ]:
wf_analog_names_path = os.path.join(notebook_output_configs_dir_wf, "dict_analog_names_sample.json") # Используем скопированный
wf_analog_content = { "bus1": { 
    "U_Active": ["U_Active"], "I_Stable": ["I_Stable"], 
    "U_Active_Sub": ["U_Active_Sub"], "U_Archived": ["U_Archived"]
} }
# Перезаписывать стандартный файл не будем, используем тот, что скопирован в notebook_output_configs_dir_wf
# with open(wf_analog_names_path, "w", encoding="utf-8") as f: json.dump(wf_analog_content, f)
wf_discrete_names_path = os.path.join(notebook_output_configs_dir_wf, "dict_discrete_names_sample.json") # Используем скопированный
wf_discrete_content = { "bus1": { "ML_Event": ["ML_Event_Dummy"]}}
# with open(wf_discrete_names_path, "w", encoding="utf-8") as f: json.dump(wf_discrete_content, f)

if not os.path.exists(workflow_norm_csv_path):
    print(f"Файл {workflow_norm_csv_path} не найден! Пропускаем этап конвертации в CSV.")
else:
    normalizer_workflow = OscillogramNormalizer(norm_coef_file_path=workflow_norm_csv_path, is_print_message=False)

    converter_wf = OscillogramToCsvConverter(
        normalizer=normalizer_workflow,
        raw_path=staging_dir_active_wf, 
        csv_path=notebook_output_configs_dir_wf, 
        # uses_buses=['1'], # Опционально, если нужно указать конкретные шины
        dict_analog_names_path=wf_analog_names_path,
        dict_discrete_names_path=wf_discrete_names_path,
        is_print_message=False,
        show_progress_bars=True
    )
    converter_wf.number_periods = 1 

    final_csv_name = "final_dataset.csv"
    print(f"\nЗапуск финального преобразования в CSV (выход: {final_csv_name})...")
    df_final = converter_wf.create_csv(csv_name=final_csv_name, is_cut_out_area=True, is_simple_csv=False)

    if df_final is not None and not df_final.empty:
        print(f"\nПервые строки итогового CSV ({final_csv_name}):")
        try: from IPython.display import display; display(df_final.head())
        except ImportError: print(df_final.head())
        print(f"Колонки: {df_final.columns.tolist()}")
    else:
        print(f"Итоговый CSV не был создан или пуст.")

## Этап 7: Специализированный анализ (Пример: `SPEFAnalyzer`)
Это опциональный шаг. Можно запустить любой из анализаторов на полученных данных.
Для `SPEFAnalyzer` потребовалась бы более специфичная настройка `norm.csv` и конфигурации самого анализатора.
Здесь просто обозначим возможность.

In [ ]:
print("\nПример вызова SPEFAnalyzer (закомментирован, требует своей конфигурации):")
# spef_config_wf = { 
# 'VALID_NOMINAL_VOLTAGES': {6000.0/np.sqrt(3)}, # Пример
# 'SPEF_THRESHOLD_U0': 0.05, 'SPEF_THRESHOLD_Un': 0.03, 
# 'SPEF_MIN_DURATION_PERIODS': 1, 
# 'SIMILAR_AMPLITUDES_FILTER_ENABLED': True, 
# 'SIMILAR_AMPLITUDES_MAX_RELATIVE_DIFFERENCE': 0.15, 
# 'verbose': True, 'norm_yes_phrase': 'YES' 
# }
# if os.path.exists(workflow_norm_csv_path):
#     norm_coeffs_df_wf = pd.read_csv(workflow_norm_csv_path)
#     spef_analyzer_wf = SPEFAnalyzer(
#         config=spef_config_wf,
#         normalizer=normalizer_workflow, # Уже есть
#         bus_splitter=converter_wf,    # Уже есть (OscillogramToCsvConverter)
#         norm_coef_df=norm_coeffs_df_wf
#     )
#     spef_report_path_wf = os.path.join(notebook_output_configs_dir_wf, "workflow_spef_report.csv")
#     spef_error_log_wf = os.path.join(notebook_output_configs_dir_wf, "workflow_spef_errors.log")
#     spef_analyzer_wf.analyze_directory(staging_dir_active_wf, spef_report_path_wf, spef_error_log_wf)
#     if os.path.exists(spef_report_path_wf):
#         print("Отчет SPEF:")
#         try: from IPython.display import display; display(pd.read_csv(spef_report_path_wf))
#         except ImportError: print(pd.read_csv(spef_report_path_wf))
# else:
#     print("Файл norm.csv не найден, анализ SPEF пропущен.")

## Этап 8: Очистка

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(notebook_base_workflow_dir):
        shutil.rmtree(notebook_base_workflow_dir)
        print(f"\nВременная директория {notebook_base_workflow_dir} удалена.")
    else:
        print(f"\nВременная директория {notebook_base_workflow_dir} не найдена для удаления.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_base_workflow_dir}: {e}")